# Applied Project in Big Data on Industrial Dataset

## MODELS SELECTION TECHNIQUES
## Part I. Vanilla pipeline

### 1. Libraries

In [ ]:
import os
import re
import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing import Pool
from sklearn.feature_extraction.text import (
    TfidfVectorizer, 
    CountVectorizer
)
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score, 
    confusion_matrix, 
    precision_score, 
    recall_score, 
    f1_score,
    roc_curve, 
    auc,
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_recall_curve,
    PrecisionRecallDisplay
)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    cross_val_score, 
    train_test_split,
    StratifiedKFold
)
pd.set_option('display.max_columns', None)
N_CORES = min(
    multiprocessing.cpu_count(), 
    int(float(os.environ['CPU_LIMIT']))
)
print('cores:', N_CORES)

### 2. Create a small dataset for demo

In [ ]:
SAMPLE_SIZE = 1000

In [ ]:
df = pd.read_csv('data_white_en.csv')
df = df.sample(SAMPLE_SIZE).reset_index()
del df['index']
df['label'] = 0
print(df.shape)
display(df.head())

In [ ]:
df_tmp = pd.read_csv('data_black_en.csv')
df_tmp = df_tmp.sample(SAMPLE_SIZE).reset_index()
del df_tmp['index']
df_tmp['label'] = 1
print(df_tmp.shape)
display(df_tmp.head())

In [ ]:
df = df.append(df_tmp)
df.reset_index(inplace=True)
del df['index']
print(df.shape)
display(df.head())

### 3. Just make a model

In [ ]:
MAX_DF = .95
MIN_DF = 8

In [ ]:
def text_features(data, vectorizer):
    print('total texts:', len(data))
    features = vectorizer.fit_transform(data)
    print(
        'features shape:', features.shape, 
        'max:', np.max(features), 
        'min:', np.min(features)
    )
    return features, vectorizer

In [ ]:
%%time
vectorizer=TfidfVectorizer(
    ngram_range=(1, 1), 
    max_df=MAX_DF, 
    min_df=MIN_DF
)
features, vectorizer = text_features(
    df['proc'], 
    vectorizer=vectorizer
)

In [ ]:
print(features.todense())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['proc'], 
    df['label'], 
    test_size=.3, 
    random_state=2022
)
X_train, vectorizer = text_features(
    X_train, 
    vectorizer=vectorizer
)
X_test = vectorizer.transform(X_test)
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
clf.get_params()

In [ ]:
y_pred

In [ ]:
roc_auc_score(y_test, y_pred)

In [ ]:
y_score = clf.predict_proba(X_test)

In [ ]:
# Scikit-learn Example of Receiver Operating Characteristic (ROC) 
# metric to evaluate classifier output quality.
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

fpr, tpr, thresholds = roc_curve(y_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)
roc_auc

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
plt.figure()
lw = 2
plt.plot(
    fpr,
    tpr,
    color='darkorange',
    lw=lw,
    label='ROC curve (area = %0.2f)' % roc_auc,
)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc='lower right')
plt.show()

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
cm

In [ ]:
prec, recall, _ = precision_recall_curve(
    y_test,
    y_score[:, 1], 
    pos_label=clf.classes_[1]
)
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot()

### 4. More advanced model approach: cross-validation

In [ ]:
def cross_val_model(X, y, 
                    folds, clf,
                    vectorizer, ngram_range=(1, 1), 
                    max_df=.2, min_df=8, seed=2022):
    scores = []
    skf = StratifiedKFold(n_splits=folds, random_state=seed, shuffle=True)
    for fold, (train_idxs, test_idxs) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_idxs], X.iloc[test_idxs]
        y_train, y_test = y.iloc[train_idxs], y.iloc[test_idxs]
        X_train, vectorizer = text_features(
            X_train, 
            vectorizer=vectorizer
        )
        X_test = vectorizer.transform(X_test)
        clf.fit(X_train, y_train)
        y_score = clf.predict_proba(X_test)
        scores_fold = roc_auc_score(y_test, y_score[:, 1])
        print(f'fold {fold} val score: {scores_fold:.2f}')
        scores.append(scores_fold)
    return scores

In [ ]:
vectorizer=TfidfVectorizer(
    ngram_range=(1, 1), 
    max_df=MAX_DF, 
    min_df=MIN_DF
)
cross_val_model(
    X=df['proc'], 
    y=df['label'], 
    folds=5, 
    clf=LogisticRegression(),
    vectorizer=vectorizer,
    seed=2022
)

### 4. Straight way to organize a project and track experiments

### 5. WandB